# Final model prediction on test set

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import random

In [2]:
import cv2
import matplotlib.pyplot as plt 

In [3]:
from sklearn.metrics import accuracy_score

In [4]:
PATH = Path("/home/ubuntu/data/sandwich/")

In [5]:
# list(PATH.iterdir())

In [6]:
def read_image(path):
    im = cv2.imread(str(path))
    return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

# Data augmentation

In [7]:
import math
def crop(im, r, c, target_r, target_c): return im[r:r+target_r, c:c+target_c]


# random crop to the original size
def random_crop(x, r_pix=8):
    """ Returns a random crop"""
    r, c,*_ = x.shape
    r, c,*_ = x.shape
    c_pix = round(r_pix*c/r)
    rand_r = random.uniform(0, 1)
    rand_c = random.uniform(0, 1)
    start_r = np.floor(2*rand_r*r_pix).astype(int)
    start_c = np.floor(2*rand_c*c_pix).astype(int)
    return crop(x, start_r, start_c, r-2*r_pix, c-2*c_pix)

def center_crop(x, r_pix=8):
    r, c,*_ = x.shape
    c_pix = round(r_pix*c/r)
    return crop(x, r_pix, c_pix, r-2*r_pix, c-2*c_pix)


def rotate_cv(im, deg, mode=cv2.BORDER_REFLECT, interpolation=cv2.INTER_AREA):
    """ Rotates an image by deg degrees"""
    r,c,*_ = im.shape
    M = cv2.getRotationMatrix2D((c/2,r/2),deg,1)
    return cv2.warpAffine(im,M,(c,r), borderMode=mode, 
                          flags=cv2.WARP_FILL_OUTLIERS+interpolation)

# Dataset

In [8]:
def get_files(path):
    paths = [d for d in list(path.iterdir()) if d.is_dir()]
    files = [f for d in paths for f in list(d.iterdir())]
    return files

In [9]:
path = PATH/"test-315"

In [10]:
test_files = get_files(path)

In [11]:
def normalize(im):
    """Normalizes images with Imagenet stats."""
    imagenet_stats = np.array([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])
    return (im - imagenet_stats[0])/imagenet_stats[1]

In [12]:
paths = [d for d in list(path.iterdir()) if d.is_dir()]

In [13]:
labels = [p.parts[-1] for p in paths]

In [14]:
labels

['hot_dog',
 'pulled_pork_sandwich',
 'grilled_cheese_sandwich',
 'lobster_roll_sandwich',
 'hamburger',
 'club_sandwich']

In [15]:
label2ind = {v:k for k,v in enumerate(labels)}

In [16]:
label2ind

{'hot_dog': 0,
 'pulled_pork_sandwich': 1,
 'grilled_cheese_sandwich': 2,
 'lobster_roll_sandwich': 3,
 'hamburger': 4,
 'club_sandwich': 5}

In [17]:
class SandwichDataset(Dataset):
    def __init__(self, files, labels, transforms=False):
        self.files = files
        self.label2ind = {v:k for k,v in enumerate(labels)}
        self.transforms = transforms
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        path = self.files[idx]
        name = path.parts[-1]
        y_class = self.label2ind[path.parts[-2]]
        x = cv2.imread(str(path)).astype(np.float32)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)/255
        if self.transforms:
            rdeg = (np.random.random()-.50)*20
            x = rotate_cv(x, rdeg)
            if np.random.random() > 0.5: x = np.fliplr(x).copy()
            x = random_crop(x)
        else:
            x = center_crop(x)
        x = normalize(x)
        y = self.label2ind[path.parts[-2]]
        return np.rollaxis(x, 2), y

# Load final best model

In [18]:
class ResnetV2(nn.Module):
    def __init__(self):
        super(ResnetV2, self).__init__()
        self.resnet = models.resnet34(pretrained=True)
        self.freeze()
        layers = list(self.resnet.children())[:8]
        self.groups = nn.ModuleList([nn.Sequential(*h) for h in [layers[:6], layers[6:]]]) # Define groups of layers
        self.groups.append(nn.Linear(512, 6))
    
    def forward(self, x):
        for group in self.groups[:2]: # Resnet layers
            x = group(x)
        x = F.relu(x)
        x = nn.AdaptiveAvgPool2d((1,1))(x)
        x = x.view(x.shape[0], -1)
        x = self.groups[2](x) # Linear layer
        return x
    
    def freeze(self): # Freeze all Resnet
        for param in self.resnet.parameters():
            param.requires_grad = False
            
    def unfreeze(self, group_idx:int): # Unfreeze a group
        group = self.groups[group_idx]
        parameters = filter(lambda x: hasattr(x,'requires_grad'), group.parameters())
        for p in parameters: 
            p.requires_grad = True

In [19]:
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))

In [20]:
model_path = Path("/home/ubuntu/models/sandwich/")

In [21]:
model = ResnetV2().cuda()

In [22]:
load_model(model, model_path/"ft_resnet_85.pth")

# Without Test Time Augmentation

In [23]:
test_ds = SandwichDataset(files=test_files, labels=labels, transforms=False)

In [24]:
# test_ds.label2ind

In [25]:
batch_size = 50

In [26]:
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [27]:
def test_arrays(model, test_dl):
    model.eval()
    preds = []
    ys = []
    for x, y in test_dl:
        x = x.cuda().float()
        out = model(x)
        _, pred = torch.max(out, 1)
        preds.append(pred.cpu().numpy())
        ys.append(y)
    return np.vstack(preds), np.vstack(ys)

In [28]:
preds, ys = test_arrays(model, test_dl)

In [29]:
preds = preds.reshape(-1)

In [30]:
ys = ys.reshape(-1)

In [31]:
accuracy_score(y_true=ys, y_pred=preds)

0.8613333333333333

# With Test Time Augmentation

In [32]:
test_ds_aug = SandwichDataset(files=test_files, labels=labels, transforms=True)

In [33]:
batch_size = 50

In [34]:
test_dl_aug = DataLoader(test_ds_aug, batch_size=batch_size, shuffle=False)

In [35]:
preds_aug, ys_aug = test_arrays(model, test_dl_aug)

In [36]:
preds_aug = preds_aug.reshape(-1)

In [37]:
ys_aug = ys_aug.reshape(-1)

In [38]:
accuracy_score(y_true=ys_aug, y_pred=preds_aug)

0.8733333333333333